In [232]:
import random
import nltk
import re

from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from nltk.classify import ClassifierI
from statistics import mode

from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
#stemmer = SnowballStemmer("english")
stemmer = PorterStemmer()
#stemmer = WordNetLemmatizer()

In [233]:
def build_model(optimizer='adam', init='glorot_normal', epochs=200, batch_size=200):
    model = Sequential()
    model.add(Dense(1000, input_dim=200, kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(750, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(500, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(250, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(125, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(25, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    # Fit the model
    #model.fit(X_train, Y_train, epochs=50, batch_size=100)
    
    return model

    

In [234]:
def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    #stems = [stemmer.lemmatize(t) for t in filtered_tokens]
    return stems

In [235]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from numpy.random import seed
import pandas as pd
from sklearn import preprocessing

labels = []
X = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()]
y = [movie_reviews.categories(fileid)[0] for fileid in movie_reviews.fileids()]
for i in y:
    if i == 'pos':
        labels.append(1)
    else:
        labels.append(0)
#random.shuffle(documents)

#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, labels, train_size = 0.9, random_state = seed(2017))
vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=True, ngram_range=(1,4), tokenizer=tokenize, stop_words='english')
X_result_train = vectorizer.fit_transform(X_train)
X_result_test = vectorizer.transform(X_test)


In [236]:
from sklearn.decomposition import TruncatedSVD 

X_train_svd = X_result_train.todense()
X_test_svd = X_result_test.todense()
ncol = X_train_svd.shape[1]
print(ncol)

svd = TruncatedSVD(n_components=200, random_state=0)
X_train_svd = svd.fit_transform(X_train_svd)
X_test_svd = svd.transform(X_test_svd)
#print(X_train_svd[4])
#print(len(X_train_svd[4]))

20849


In [237]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neural_network import MLPClassifier

model = build_model()
model.fit(X_train_svd, Y_train, epochs=50, batch_size=250)

Epoch 1/50
1800/1800 [==============================] - 3s - loss: 0.6927 - acc: 0.5006     
Epoch 2/50
1800/1800 [==============================] - 1s - loss: 0.6664 - acc: 0.5511     
Epoch 3/50
1800/1800 [==============================] - 1s - loss: 0.4788 - acc: 0.8311     
Epoch 4/50
1800/1800 [==============================] - 1s - loss: 0.3149 - acc: 0.8844     
Epoch 5/50
1800/1800 [==============================] - 1s - loss: 0.2961 - acc: 0.8956     
Epoch 6/50
1800/1800 [==============================] - 1s - loss: 0.2044 - acc: 0.9289     
Epoch 7/50
1800/1800 [==============================] - 1s - loss: 0.1535 - acc: 0.9456     
Epoch 8/50
1800/1800 [==============================] - 1s - loss: 0.1251 - acc: 0.9622     
Epoch 9/50
1800/1800 [==============================] - 1s - loss: 0.1077 - acc: 0.9656     
Epoch 10/50
1800/1800 [==============================] - 1s - loss: 0.0990 - acc: 0.9672     
Epoch 11/50
1800/1800 [==============================] - 1s - loss: 0

In [239]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier

#eclf3 = GradientBoostingClassifier(n_estimators=1000, random_state=5)
#eclf3.fit(X_train_svd, Y_train)

combined_features_test1 = model.predict(X_test_svd)
#combined_features_test1 = eclf3.predict(X_test_svd)
combined_features_test1 = np.rint(combined_features_test1)

print(accuracy_score(Y_test, combined_features_test1) * 100)

88.0
